In [1]:
import MySQLdb
import csv
import os
import subprocess
import pandas as pd
import numpy as np

In [2]:
def write_header():
    head = ["cid","time","compile","vmlinux","GZIP-bzImage","GZIP-vmlinux","GZIP","BZIP2-bzImage","BZIP2-vmlinux","BZIP2","LZMA-bzImage","LZMA-vmlinux","LZMA","XZ-bzImage","XZ-vmlinux","XZ","LZO-bzImage","LZO-vmlinux","LZO","LZ4-bzImage","LZ4-vmlinux","LZ4"]
    with open("stats.csv", 'w') as file:
        writer = csv.DictWriter(file,head)
        writer.writeheader()

In [3]:
def kernel2csv(compressed):
    if compressed is None:
        return [str(-1)] * 18
    
    compressed = compressed.split(" , ")
    parse_array = [i.split(" : ") for i in compressed]
    final = [fin[1] for fin in parse_array]

    return final

In [4]:
## Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [5]:
def convert_time(value):
    unit = "secondes"
    value = float(value)
    if value >= 60.0 and unit == "secondes":
        value = value/60.0
        unit = "minutes"
    
    if value >= 60.0 and unit == "minutes":
        value = value/60.0
        unit = "hours"
        
    if value >= 24 and unit == "hours":
        value = value / 24.0
        unit = "days"

    tmp = str(value).split(".")
    value = tmp[0] + "." + tmp[1][:2]
        
    return value, unit

In [6]:
def convert_size(value):
    unit = "o"
    value = float(value)
    if value >= 1000.0 and unit == "o":
        value = value / 1000.0
        unit = "ko"
    
    if value >= 1000.0 and unit == "ko":
        value = value / 1000.0
        unit = "Mo"
    
    if value >= 1000.0 and unit == "Mo":
        value = value / 1000.0
        unit = "Go"
        
    tmp = str(value).split(".")
    value = tmp[0] + "." + tmp[1][:3]
    
    return value, unit

In [7]:
def min_max():
    
    df = pd.read_csv('stats.csv')
    ok_or_not = True
    
    size_max = max(df["vmlinux"])
    size_min = min(df["vmlinux"].where(df["compile"] == ok_or_not))
    tmp_size_average = df["vmlinux"].where(df["compile"] == ok_or_not).mean()
    size_average = float(str(tmp_size_average).split('.')[0] + '.' + str(tmp_size_average).split('.')[1][:3])
    
    size_max, size_max_unit = convert_size(size_max)
    size_min, size_min_unit = convert_size(size_min)
    size_average, size_average_unit = convert_size(size_average)
    
    time_max = max(df["time"])
    time_min = min(df["time"].where(df["compile"] == ok_or_not))
    tmp_time_average = df["time"].where(df["compile"] == ok_or_not).mean()
    time_average = float(str(tmp_time_average).split('.')[0] + '.' + str(tmp_time_average).split('.')[1][:3])
    
    time_max, time_max_unit = convert_time(time_max)
    time_min, time_min_unit = convert_time(time_min)
    time_average, time_average_unit = convert_time(time_average)
    
    print("Number of values:", len(df))
    
    print("Time Max:", time_max, time_max_unit, end="")
    print(", Min:", time_min, time_min_unit, end="")
    print("\taverage:", time_average, time_average_unit)
    
    print("Size Max:", size_max, size_max_unit, end="")
    print(", Min:", size_min, size_min_unit, end="")
    print("\t\taverage:", size_average, size_average_unit)
    
    return df
        

In [8]:
def succeed(entry):

    for i in range(len(entry)-1):
        if "-1" in str(entry[i]) or "-2" in str(entry[i]) or "-3" in str(entry[i]):
            return [False]
    
    err = bz2.decompress(entry[4]).decode().split("\n")
    #if "make: *** [gcc-plugins-check] Error 1" in err:
    for line in err:
        if re.match("make: \*\*\*( \[[\w'-]*\])* Error \d+", line):
            return [False]

    return [True]

In [9]:
def create_csv():
    
    subprocess.run("rm stats.csv", shell=True)
    if not os.path.exists("stats.csv"):
        write_header()  
        
    entry_unformatted = compute_rows()
    entry = []
    for i in range(len(entry_unformatted)):
        compil = succeed(entry_unformatted[i])
        compr = kernel2csv(entry_unformatted[i][3])
        entry.append(list(entry_unformatted[i][0: 2]) + compil + [entry_unformatted[i][2]] + compr)
    
    with open("stats.csv", 'a') as file:
        writer = csv.writer(file)
        for line in entry:
            writer.writerow(line)
    subprocess.run('wc -c stats.csv', shell=True)
    

In [10]:
def compute_rows():
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT cid, compilation_time, core_size, compressed_sizes FROM Compilations WHERE cid >= 55000 ORDER BY cid DESC"
        cursor.execute(query)
        entry = cursor.fetchall()
    except:
        print("Error")
    finally:
        cursor.close()
        socket.close()
    
    return entry

In [11]:
def main():  
    create_csv()
    min_max()
    

In [12]:
main()

22 
 ['cid', 'time', 'compile', 'vmlinux', 'GZIP-bzImage', 'GZIP-vmlinux', 'GZIP', 'BZIP2-bzImage', 'BZIP2-vmlinux', 'BZIP2', 'LZMA-bzImage', 'LZMA-vmlinux', 'LZMA', 'XZ-bzImage', 'XZ-vmlinux', 'XZ', 'LZO-bzImage', 'LZO-vmlinux', 'LZO', 'LZ4-bzImage', 'LZ4-vmlinux', 'LZ4']
